# rcsbsearchapi

Access the RCSB advanced search from python: [rcsbsearchapi.readthedocs.io](https://rcsbsearchapi.readthedocs.io)

    pip install rcsbsearchapi
    
## Demo

We are interested in how the antiviral drug boceprevir interacts with the Covid-19 virus. 
- Source Organism is "COVID-19 virus"
- Structure title contains "protease"
- Bound to ligand "Boceprevir"
- With resolution at least 1.5

[RCSB Query](https://www.rcsb.org/search?request=%7B%22query%22%3A%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22full_text%22%2C%22parameters%22%3A%7B%22value%22%3A%22protease%22%7D%7D%5D%2C%22logical_operator%22%3A%22and%22%7D%5D%2C%22label%22%3A%22full_text%22%7D%2C%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text_chem%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22chem_comp.name%22%2C%22operator%22%3A%22contains_words%22%2C%22negation%22%3Afalse%2C%22value%22%3A%22Boceprevir%22%7D%7D%5D%2C%22logical_operator%22%3A%22and%22%7D%5D%2C%22label%22%3A%22text_chem%22%7D%2C%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_entity_source_organism.taxonomy_lineage.name%22%2C%22operator%22%3A%22exact_match%22%2C%22negation%22%3Afalse%2C%22value%22%3A%22COVID-19%20virus%22%7D%7D%5D%2C%22logical_operator%22%3A%22and%22%7D%2C%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_entry_info.diffrn_resolution_high.value%22%2C%22operator%22%3A%22greater%22%2C%22negation%22%3Afalse%2C%22value%22%3A1.5%7D%7D%5D%2C%22logical_operator%22%3A%22and%22%7D%5D%2C%22label%22%3A%22text%22%7D%5D%7D%2C%22return_type%22%3A%22entry%22%2C%22request_options%22%3A%7B%22paginate%22%3A%7B%22start%22%3A0%2C%22rows%22%3A25%7D%2C%22results_content_type%22%3A%5B%22experimental%22%5D%2C%22sort%22%3A%5B%7B%22sort_by%22%3A%22score%22%2C%22direction%22%3A%22desc%22%7D%5D%2C%22scoring_strategy%22%3A%22combined%22%7D%2C%22request_info%22%3A%7B%22query_id%22%3A%224a448274ca176680bee98f84fa3614be%22%7D%7D)

In [ ]:
from rcsbsearchapi import rcsb_attributes as attrs, TextQuery
from rcsbsearchapi.search import Facet
import nglview

## Operator syntax
- Uses python comparison operators for basic attributes (`==`, `<`, `<=`, etc)
- Combine using set operators (`&`, `|`, `~`, etc)
- Execute queries as functions

In [ ]:
q1 = attrs.rcsb_entity_source_organism.taxonomy_lineage.name == "COVID-19 virus"
q2 = TextQuery("protease")
q3 = attrs.chem_comp.name.contains_words("Boceprevir")
q4 = attrs.rcsb_entry_info.resolution_combined > 1.5
query = q1 & q2 & q3 & q4

# Categorize results by release year
print(query.facets(facets=Facet(name="Release Date", aggregation_type="date_histogram", attribute="rcsb_accession_info.initial_release_date", interval="year")))

list(query(results_verbosity="minimal"))

In [ ]:
nglview.show_pdbid('7brp')

## Fluent syntax

A second syntax is available with a [fluent interface](https://en.wikipedia.org/wiki/Fluent_interface), similar to popular data science packages like tidyverse and Apache Spark. Function calls  are chained together.

Here's an example around a second antiviral, remdesivir. The drug interferes with RNA polymerase, replacing an adenine and causing early chain termination. When integrated into RNA, the nucleotide formed from remdesivir has residue code F86.

In [ ]:
attrs.struct.title.contains_phrase("RNA polymerase")\
    .or_(attrs.struct.title).contains_words("RdRp")\
    .and_(attrs.rcsb_entity_source_organism.taxonomy_lineage.name).exact_match("COVID-19 virus")\
    .and_(attrs.rcsb_chem_comp_container_identifiers.comp_id).exact_match("F86")\
    .exec()\
    .iquery()


In [ ]:
view = nglview.show_pdbid('7B3C', default_representation=False)
#view.get_state()['_camera_orientation']
o = [6, 3, 23, 0, 23, 1, -6, 0, -2, 24, -2, 0, -84, -92, -109, 1]
view.control.orient(o)
view.add_surface(sele="protein", opacity=.8, color="electrostatic")
view.add_cartoon(sele="rna", color="cyan")
view.add_licorice(sele="rna", color="cyan")
view.add_spacefill(sele="F86")
view

## Try it!

[rcsbsearchapi.readthedocs.io](rcsbsearchapi.readthedocs.io)